## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [2]:
#configure directory to import sast libraries
import sys 
import os 
#add sast library path
file_path = os.path.expanduser("~/random_sast/sast")
sys.path.append(file_path)
#add cd_diagram library path
file_path = os.path.expanduser("~/random_sast/cd_diagram")
sys.path.append(file_path)

#sys.path

In [3]:
from sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
import os

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/mass_ts/_mass_ts.py:17: UserWarning: GPU support will not work. You must pip install mass-ts[gpu].
  warnings.warn(


### Reading Datasets

In [4]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [5]:
df_other_methods['method'].unique()

array(['ROCKET', 'HIVE-COTEv1', 'BOSS', 'RISE', 'TSF', 'InceptionTime',
       'TS-CHIEF', 'S-BOSS', 'STC', 'ResNet', 'ProximityForest', 'WEASEL',
       'Catch22', 'cBOSS'], dtype=object)

In [24]:
# Set directory where the csv files are located
#directory = os.getcwd()+'/ResultsRsast'
directory = os.getcwd()+'/results_accuracy_per_ds'
# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.startswith("df_overall_tunning") and filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)



In [25]:
#df_result=pd.read_csv("df_overall_tunning_SmoothSubspace3.csv")
df_result['dataset_name'].unique()


array(['Chinatown'], dtype=object)

In [26]:
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [14]:
# create a pivot table with the mean of score by dataset
len_method_vs_ds=pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="STC"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('method',axis=1)
merged_df

,dataset_name,ACF,ACF&PACF,Max ACF,Max PACF,None,PACF,dataset,score
0,Chinatown,0.914723,0.934888,0.92598,0.940152,0.930272,0.941772,Chinatown,0.96793


In [32]:
# create a pivot table with the mean of score by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,1,10
len_method,,
ACF,0.906706,0.922741
ACF&PACF,0.911889,0.957888
Max ACF,0.929543,0.922417
Max PACF,0.926466,0.953839
None,0.925170,0.935374
PACF,0.917881,0.965662


In [33]:
# create a pivot table with the mean of score by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,(max instances per class)//2,1,10
len_method,,,,,
ACF,0.719631,0.969388,0.951409,0.930515,0.965986
ACF&PACF,0.838678,0.964529,0.940962,0.944121,0.980078
Max ACF,0.882410,0.965015,0.950437,0.838192,0.969388
Max PACF,0.887269,0.963557,0.949708,0.909135,0.981535
None,0.867347,0.964529,0.936832,0.911079,0.965015
PACF,0.908649,0.964529,0.927843,0.940719,0.981050


#### Time: Subsequence Lenght Method 

In [77]:
# create a pivot table with the mean of the time by dataset
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['dataset_name'], aggfunc='mean')

dataset_name,BirdChicken,Chinatown,Coffee,DistalPhalanxOutlineAgeGroup,DistalPhalanxOutlineCorrect,GunPoint,ItalyPowerDemand,ShapeletSim,SmoothSubspace,SonyAIBORobotSurface1,SonyAIBORobotSurface2,SyntheticControl
len_method,,,,,,,,,,,,
ACF,81.630738,0.143859,29.515313,132.824072,176.923921,39.144550,0.826132,7.645123,2.081739,0.419210,0.872094,128.598058
ACF&PACF,297.280662,0.444094,102.533573,419.191777,507.184570,88.200498,2.032732,222.211859,3.407822,2.247746,3.368165,402.314136
Max ACF,2.745482,0.145839,2.160797,49.805655,58.944910,4.153232,0.839920,2.640422,2.087500,0.402863,0.671550,45.391469
Max PACF,107.885490,0.162721,6.330146,29.798286,35.627314,3.936128,0.915300,124.217464,2.404325,0.405555,0.545936,29.918500
None,1.853694,0.133453,1.236922,28.495936,34.739709,2.529203,0.829915,1.836391,2.064745,0.300002,0.440205,28.232846
PACF,208.928276,0.387296,74.365190,293.491555,341.327012,48.907430,2.043757,215.296044,3.382209,1.952652,2.593810,260.544206


In [78]:
# create a pivot table with the mean of the time by number of instances per class
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
len_method,,,,,
ACF,2.339472,23.148029,45.665547,62.839905,83.037665
ACF&PACF,10.123854,96.820569,169.159711,216.207651,278.284560
Max ACF,0.541041,4.597234,12.020609,18.039164,25.389061
Max PACF,2.936125,28.781154,37.718870,37.069176,40.764452
None,0.386081,2.945729,7.396760,10.896400,15.269686
PACF,7.444393,73.907325,119.458201,154.299323,196.183556


In [79]:
# create a pivot table with the mean of the time by number of random points
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,1,10,30,50,100
len_method,,,,,
ACF,1.289364,11.672731,34.182388,56.712668,113.173468
ACF&PACF,24.724018,55.801325,126.254698,202.023784,361.792521
Max ACF,0.433520,3.307829,9.519967,15.867181,31.458614
Max PACF,20.715926,21.932824,29.043358,33.092904,42.484765
None,0.280917,2.025245,5.855033,9.372271,19.361188
PACF,23.283380,43.563970,94.274854,139.454429,250.716165


#### Variance: Subsequence Lenght Method 

In [80]:
# create a pivot table with the variance of the accuracy by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,1,10,30,50,100
len_method,,,,,
ACF,0.043978,0.047734,0.040993,0.039286,0.038609
ACF&PACF,0.044760,0.048976,0.041255,0.042711,0.039714
Max ACF,0.054232,0.047649,0.049929,0.043011,0.046338
Max PACF,0.060484,0.048674,0.055453,0.051539,0.049707
None,0.058004,0.053108,0.055950,0.049005,0.048956
PACF,0.040553,0.047795,0.037283,0.041249,0.043823


In [81]:
# create a pivot table with the variance of the accuracy by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='var')

rpoint,1,10,30,50,100
len_method,,,,,
ACF,0.052706,0.042806,0.042890,0.037358,0.035869
ACF&PACF,0.051362,0.043846,0.042672,0.040587,0.039867
Max ACF,0.054459,0.048488,0.059475,0.042335,0.042084
Max PACF,0.066049,0.048683,0.060449,0.047711,0.045021
None,0.065074,0.051135,0.055476,0.050537,0.047541
PACF,0.045751,0.048991,0.039095,0.041778,0.035648


#### Another Results

In [82]:
df_result[['dataset_name','len_method','classifier_name']].describe()

,dataset_name,len_method,classifier_name
count,4800,4800,4800
unique,12,6,150
top,SmoothSubspace,ACF&PACF,ACF&PACF: n_random_points=1 nb_inst_per_class=1
freq,450,800,32


In [83]:
pd.pivot_table(df_result, values='time', index=['dataset_name'], aggfunc='sum')/(60*60)

,time
dataset_name,
BirdChicken,14.590090
Chinatown,0.029526
Coffee,1.500986
DistalPhalanxOutlineAgeGroup,19.866818
DistalPhalanxOutlineCorrect,8.019079
GunPoint,3.893147
ItalyPowerDemand,0.155995
ShapeletSim,11.955152
SmoothSubspace,0.321424


In [84]:
df_result['time'].sum()/(60*60)

79.274280236827

#### Generate CD Diagram Tunning Hyperparameter

In [85]:
#create dataset to generate cd diagram
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
#df_perf.classifier_name.unique()

In [86]:
from cd_function import *
#generate hyperparameter tuning cd diagrams
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF&PACF:')], labels=True, title="ACF&PACF comparison", name_file='images_cd_diagram/cd-diagram_ACF&PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^PACF:')], labels=True, title="PACF comparison", name_file='images_cd_diagram/cd-diagram_PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF:')], labels=True, title="ACF comparison", name_file='images_cd_diagram/cd-diagram_ACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^None:')], labels=True, title="None comparison", name_file='images_cd_diagram/cd-diagram_None.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max ACF:')], labels=True, title="Max ACF comparison", name_file='images_cd_diagram/cd-diagram_MaxACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max PACF:')], labels=True, title="Max PACF comparison", name_file='images_cd_diagram/cd-diagram_MaxPACF.png')
draw_cd_diagram(df_perf, labels=True, title="overall Hyperparameter Tunning", name_file='images_cd_diagram/cd-diagram_overall_tunning.png')



/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not 

In [87]:
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)
average_ranks

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


,0
Max PACF: n_random_points=30 nb_inst_per_class=1,127.916667
Max ACF: n_random_points=30 nb_inst_per_class=1,123.250000
Max ACF: n_random_points=1 nb_inst_per_class=1,121.875000
Max ACF: n_random_points=10 nb_inst_per_class=1,117.541667
Max PACF: n_random_points=1 nb_inst_per_class=30,116.666667
...,...
ACF&PACF: n_random_points=100 nb_inst_per_class=50,42.750000
ACF&PACF: n_random_points=100 nb_inst_per_class=30,42.375000
PACF: n_random_points=100 nb_inst_per_class=30,41.375000
ACF&PACF: n_random_points=10 nb_inst_per_class=100,41.125000


In [88]:
#generate example cd diagrams
draw_cd_diagram(pd.read_csv('example_cd_diagram.csv'), labels=True, title="Example CD Diagram", name_file='cd-diagram_example.png')

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


#### Focus on most accurate lenght method

##### Overall Accuracy

In [89]:
# highest accurate hyperparameters
average_ranks[average_ranks[0]==min_rank][0].index

Index(['ACF&PACF: n_random_points=50 nb_inst_per_class=100'], dtype='object')

In [90]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="ACF&PACF"]
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.716990,0.707127,0.813616,0.809033,0.813090
10,0.736965,0.814283,0.818863,0.814046,0.820416
30,0.767948,0.805414,0.816957,0.816813,0.819020
50,0.807897,0.811014,0.800429,0.821346,0.825309
100,0.800016,0.809959,0.818897,0.818032,0.815656


In [91]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.066533,0.080303,0.030073,0.040424,0.033693
10,0.062014,0.033533,0.039029,0.043808,0.041195
30,0.041493,0.049400,0.038865,0.045356,0.041841
50,0.024748,0.038161,0.060122,0.044564,0.040154
100,0.028426,0.040994,0.043260,0.044824,0.046722


##### Overall time complexity

In [92]:
# create a pivot table with the mean of time spent in minutes
pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='mean')/60

,cweights_time,fsubsequence_time,tclassifier_time,tdataset_time,time
dataset_name,,,,,
BirdChicken,0.001775,1.845793,0.000515,3.106501,4.954678
Chinatown,0.000160,0.001823,0.000126,0.005264,0.007402
Coffee,0.001159,0.119599,0.000517,1.587549,1.708893
DistalPhalanxOutlineAgeGroup,0.000686,0.038889,0.010279,6.936574,6.986530
DistalPhalanxOutlineCorrect,0.000761,0.028729,0.012558,8.410944,8.453076
GunPoint,0.000717,0.042117,0.001210,1.425890,1.470008
ItalyPowerDemand,0.000144,0.003278,0.000300,0.030122,0.033879
ShapeletSim,0.002136,2.069743,0.000366,1.631211,3.703531
SmoothSubspace,0.000118,0.003916,0.000706,0.052012,0.056797


In [93]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.059267,0.446878,0.526805,0.518697,0.508688
10,0.079947,0.725623,1.075959,1.264428,1.504154
30,0.137414,1.335132,2.261715,2.975149,3.811814
50,0.211412,2.012466,3.960022,4.480130,6.171286
100,0.355615,3.548282,6.272142,8.778900,11.194438


##### Calculate weights time complexity

In [94]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.037933,0.035040,0.037850,0.038401,0.041002
10,0.040538,0.036245,0.042641,0.043203,0.047068
30,0.044977,0.042565,0.046539,0.050167,0.048851
50,0.044484,0.045780,0.045468,0.048524,0.049981
100,0.050934,0.043582,0.049313,0.049783,0.046869


##### Finding subsequences time complexity

In [95]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,2.619535,24.856164,27.900273,26.070745,23.765545
10,2.788928,25.006459,28.177670,26.608351,24.043635
30,2.542850,24.268097,29.010061,31.668102,30.311706
50,3.031036,30.201905,32.043904,28.308357,27.977514
100,3.025260,28.780339,26.399535,40.387128,29.022799


##### Transform Dataset time complexity

In [96]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.878649,1.860593,3.620241,4.956047,6.655589
10,1.918754,18.417328,36.243088,49.119594,66.034704
30,5.594412,55.710952,106.496284,146.490877,196.670274
50,9.524848,90.367184,178.356234,240.140503,323.483648
100,18.173916,183.906047,348.941621,484.973417,640.888910


##### Classifier time complexity

In [97]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.017345,0.059865,0.048935,0.055273,0.057892
10,0.047579,0.076098,0.092173,0.092033,0.120973
30,0.061133,0.084119,0.145335,0.294360,1.670984
50,0.081386,0.128701,27.149624,0.301612,18.757385
100,0.082500,0.160778,0.927903,1.310779,1.693620
